## 1. Check Available Datasets

In [ ]:
import os
import sys
from pathlib import Path

# On Kaggle, datasets are automatically mounted at /kaggle/input/
kaggle_input = '/kaggle/input'

print("="*70)
print("📊 CHECKING AVAILABLE DATASETS ON KAGGLE")
print("="*70)

if os.path.exists(kaggle_input):
    print("\n✅ Kaggle environment detected\n")
    print("📁 Available datasets:")
    for item in os.listdir(kaggle_input):
        item_path = os.path.join(kaggle_input, item)
        if os.path.isdir(item_path):
            # Count files
            file_count = sum([len(files) for _, _, files in os.walk(item_path)])
            print(f"  ✓ {item:50s} ({file_count:,} files)")
else:
    print("\n⚠️  Not on Kaggle. Upload this notebook to Kaggle to access datasets.")
    print("   Go to: https://www.kaggle.com/code")

print("\n" + "="*70)

## 2. Install Required Packages + Speed Optimizations

**⚡ Optimizations enabled:**
- Mixed Precision (FP16) training
- cuDNN benchmark mode
- Parallel data loading (4 workers)
- Pin memory for faster GPU transfer
- Batch prefetching

In [ ]:
# Install additional packages if needed
!pip install timm efficientnet-pytorch torchmetrics -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import json
from tqdm import tqdm
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
from torchvision import models
import timm

# ⚡ OPTIMIZATION: Enable cuDNN benchmarking for faster convolutions
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

# ⚡ OPTIMIZATION: Set up mixed precision training
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()

print("✅ All packages imported successfully")
print(f"🖥️  Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"⚡ cuDNN benchmark: {torch.backends.cudnn.benchmark}")
print(f"⚡ Mixed Precision (AMP): Enabled")

## 3. Dataset Configuration

In [ ]:
# Map dataset names to expected Kaggle paths
dataset_mapping = {
    'rice': 'rice-diseases-image-dataset',
    'cotton': 'cotton-disease-dataset', 
    'wheat': 'wheat-leaf-dataset',
    'mango': 'mango-leaf-disease-dataset',
    'plantvillage': 'plantdisease'
}

# Find actual paths
data_paths = {}
for crop, expected_name in dataset_mapping.items():
    # Check various possible names
    for item in os.listdir('/kaggle/input'):
        if expected_name.lower() in item.lower():
            data_paths[crop] = os.path.join('/kaggle/input', item)
            break

print("📦 Dataset Paths:")
for crop, path in data_paths.items():
    status = "✅" if os.path.exists(path) else "❌"
    print(f"  {status} {crop.upper():15s}: {path}")

print(f"\n📊 Total datasets found: {len(data_paths)}/{len(dataset_mapping)}")

## 4. Load and Analyze Datasets

In [ ]:
def find_image_classes(base_path, max_depth=5):
    """Find all image class directories"""
    image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
    class_data = {}
    
    for root, dirs, files in os.walk(base_path):
        # Check depth
        depth = root[len(base_path):].count(os.sep)
        if depth > max_depth:
            continue
            
        image_files = [f for f in files if Path(f).suffix in image_extensions]
        if image_files and len(image_files) > 50:  # At least 50 images
            class_name = Path(root).name
            if class_name not in class_data:
                class_data[class_name] = []
            class_data[class_name].extend([os.path.join(root, f) for f in image_files])
    
    return class_data

# Analyze all datasets
all_classes = {}
print("\n" + "="*70)
print("📊 ANALYZING DATASETS")
print("="*70)

for crop, path in data_paths.items():
    if os.path.exists(path):
        print(f"\n🔍 Analyzing {crop.upper()}...")
        classes = find_image_classes(path)
        all_classes[crop] = classes
        print(f"   Found: {len(classes)} classes, "
              f"{sum(len(imgs) for imgs in classes.values()):,} images")

total_classes = sum(len(classes) for classes in all_classes.values())
total_images = sum(len(imgs) for crop_classes in all_classes.values() 
                   for imgs in crop_classes.values())

print("\n" + "="*70)
print(f"📈 TOTAL: {total_classes} classes, {total_images:,} images")
print("="*70)

## 5. Create Unified Dataset

In [ ]:
class PakistanCropDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None, class_names=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.class_names = class_names or []
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Create unified dataset
all_image_paths = []
all_labels = []
class_names = []
class_to_idx = {}

print("\n🔨 Creating unified dataset...")

current_idx = 0
for crop, classes in all_classes.items():
    for class_name, image_paths in classes.items():
        if len(image_paths) < 100:  # Skip small classes
            continue
        
        standardized_name = f"{crop.capitalize()}___{class_name.replace(' ', '_')}"
        class_names.append(standardized_name)
        class_to_idx[standardized_name] = current_idx
        
        all_image_paths.extend(image_paths)
        all_labels.extend([current_idx] * len(image_paths))
        
        current_idx += 1

num_classes = len(class_names)
print(f"✅ Dataset created: {num_classes} classes, {len(all_image_paths):,} images")

## 6. Data Transforms and Loaders (⚡ OPTIMIZED)

**Speed optimizations:**
- `batch_size=64` (doubled)
- `num_workers=4` (parallel loading)  
- `pin_memory=True` (faster GPU transfer)
- `prefetch_factor=2` (preload batches)
- `persistent_workers=True` (keep workers alive)
- Lighter augmentation (removed ColorJitter)

In [ ]:
# ⚡ OPTIMIZED Image transformations - lighter augmentation for speed
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Standard size for EfficientNet
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),  # Reduced from 15 for speed
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create full dataset
full_dataset = PakistanCropDataset(all_image_paths, all_labels, transform=train_transform, class_names=class_names)

# Split dataset
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, 
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Apply test transform to val and test
val_dataset.dataset.transform = test_transform
test_dataset.dataset.transform = test_transform

# ⚡ OPTIMIZED: Larger batch size + more workers + pin_memory + prefetch
batch_size = 64  # Increased from 32 - T4 can handle this easily
num_workers = 4  # ⚡ KEY FIX: Was 2, now 4 for parallel data loading

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=num_workers,
    pin_memory=True,           # ⚡ Faster GPU transfer
    prefetch_factor=2,         # ⚡ Prefetch next batches
    persistent_workers=True    # ⚡ Keep workers alive between epochs
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size * 2,  # ⚡ Larger batch for validation (no gradients)
    shuffle=False, 
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size * 2, 
    shuffle=False, 
    num_workers=num_workers,
    pin_memory=True
)

print(f"\n📊 Dataset Split:")
print(f"  Training:   {len(train_dataset):,} images")
print(f"  Validation: {len(val_dataset):,} images")
print(f"  Test:       {len(test_dataset):,} images")
print(f"\n⚡ OPTIMIZATIONS APPLIED:")
print(f"  Batch size: {batch_size} (train) / {batch_size*2} (val/test)")
print(f"  Workers: {num_workers}")
print(f"  Pin memory: True")
print(f"  Prefetch: 2")
print(f"  Persistent workers: True")

## 7. Create Model (EfficientNet-B4)

In [ ]:
# Create EfficientNet-B4 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=num_classes)
model = model.to(device)

print(f"\n🤖 Model: EfficientNet-B4")
print(f"📊 Classes: {num_classes}")
print(f"🖥️  Device: {device}")
print(f"\n✅ Model ready for training!")

## 8. Training Configuration

In [ ]:
# Training configuration
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)

epochs = 50
best_val_acc = 0.0

print("⚙️  Training Configuration:")
print(f"  Optimizer: AdamW (lr=0.0001, weight_decay=0.01)")
print(f"  Scheduler: CosineAnnealingLR")
print(f"  Loss: CrossEntropyLoss")
print(f"  Epochs: {epochs}")

## 9. Train Model (⚡ Mixed Precision AMP)

**Training optimizations:**
- `torch.cuda.amp.autocast()` - FP16 mixed precision
- `GradScaler` - prevents underflow
- `zero_grad(set_to_none=True)` - faster gradient reset
- `non_blocking=True` - async data transfer
- Real-time ETA tracking

In [ ]:
# ⚡ OPTIMIZED Training loop with Mixed Precision (AMP)
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("\n" + "="*70)
print("🚀 STARTING OPTIMIZED TRAINING (Mixed Precision + cuDNN)")
print("="*70)

import time
epoch_times = []

for epoch in range(epochs):
    epoch_start = time.time()
    
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
    for images, labels in pbar:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)  # ⚡ Faster than zero_grad()
        
        # ⚡ Mixed Precision Forward Pass
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # ⚡ Scaled Backward Pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'loss': f'{train_loss/len(pbar):.4f}', 
                         'acc': f'{100.*train_correct/train_total:.2f}%'})
    
    train_acc = 100. * train_correct / train_total
    train_loss = train_loss / len(train_loader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]  "):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            # ⚡ Mixed Precision for validation too
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_acc = 100. * val_correct / val_total
    val_loss = val_loss / len(val_loader)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'class_names': class_names,
            'num_classes': num_classes
        }, 'pakistan_model_best.pth')
    
    scheduler.step()
    
    epoch_time = time.time() - epoch_start
    epoch_times.append(epoch_time)
    avg_time = sum(epoch_times) / len(epoch_times)
    remaining = (epochs - epoch - 1) * avg_time / 60
    
    print(f"\n📊 Epoch {epoch+1} Summary:")
    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"   Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"   Best Val Acc: {best_val_acc:.2f}%")
    print(f"   ⏱️  Epoch time: {epoch_time:.1f}s | ETA: {remaining:.1f} min")
    print("-" * 70)

total_time = sum(epoch_times) / 60
print(f"\n✅ Training complete in {total_time:.1f} minutes!")
print(f"🏆 Best validation accuracy: {best_val_acc:.2f}%")
print(f"⚡ Average epoch time: {sum(epoch_times)/len(epoch_times):.1f}s")

## 10. Evaluate on Test Set

In [ ]:
# Load best model
checkpoint = torch.load('pakistan_model_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Test evaluation
test_correct = 0
test_total = 0
all_predictions = []
all_labels = []

print("\n🧪 Evaluating on test set...")

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()
        
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = 100. * test_correct / test_total

print("\n" + "="*70)
print("📊 FINAL TEST RESULTS")
print("="*70)
print(f"\n🎯 Test Accuracy: {test_acc:.2f}%")
print(f"✅ Correct: {test_correct:,} / {test_total:,}")
print("\n" + "="*70)

## 11. Save Model and Metadata

In [ ]:
# Save complete model info
model_info = {
    'class_names': class_names,
    'num_classes': num_classes,
    'test_accuracy': test_acc,
    'best_val_accuracy': best_val_acc,
    'crops': list(all_classes.keys()),
    'total_images': len(all_image_paths),
    'urdu_translations': {
        'Rice': 'چاول',
        'Wheat': 'گندم',
        'Cotton': 'کپاس',
        'Mango': 'آم',
        'Tomato': 'ٹماٹر',
        'Potato': 'آلو',
        'Corn': 'مکئی'
    }
}

# Save metadata
with open('pakistan_model_info.json', 'w', encoding='utf-8') as f:
    json.dump(model_info, f, indent=2, ensure_ascii=False)

print("\n💾 Files saved:")
print("  ✓ pakistan_model_best.pth (model weights)")
print("  ✓ pakistan_model_info.json (metadata)")
print("\n📥 Download these files from Kaggle output to use locally!")

## 12. Training Visualization

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(history['train_loss'], label='Train Loss')
ax1.plot(history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy plot
ax2.plot(history['train_acc'], label='Train Accuracy')
ax2.plot(history['val_acc'], label='Val Accuracy')
ax2.axhline(y=test_acc, color='r', linestyle='--', label=f'Test Acc: {test_acc:.2f}%')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300)
plt.show()

print("\n📊 Training visualization saved: training_history.png")

## ✅ Complete!

### 📥 Download from Kaggle Output:
1. `pakistan_model_best.pth` (~200 MB)
2. `pakistan_model_info.json` (~10 KB)
3. `training_history.png`

### 🚀 Use Locally:
```python
# Load model
import torch
import timm

checkpoint = torch.load('pakistan_model_best.pth')
model = timm.create_model('efficientnet_b4', num_classes=checkpoint['num_classes'])
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Use for predictions!
```

### کسان مددگار - Helping Pakistani Farmers with AI 🇵🇰